In [1]:
# native libraries
import numpy as np
from helpers import *

In [2]:
def predict_labels_test(weights, data):
    """Generates class predictions given weights, and a test data matrix"""
    y_pred = np.dot(data, weights)
    y_pred[np.where(y_pred <= 0)] = 0
    y_pred[np.where(y_pred > 0)] = 1

    return y_pred

In [3]:
def build_model_data_standardized(x,y):
    """Form (y,tX) to get regression data in matrix form."""
    num_samples = len(x[:,0])
    num_features = len(x[0,:])
    tx = np.ones(num_samples)
    for i in range(num_features):
        tmp,_,_ = standardize(x[:,i])
        tx = np.c_[tx, tmp] 
    tx = np.nan_to_num(tx)

    return tx, y
    
def load_build_split(loc, degree, split_ratio):
    """Load the data, build the model and split as train, test samples."""
    path = loc
    y, x, ids = load_csv_data(path)
    x[x==-999] = np.nan
    x = build_poly(x, degree)[:,1:]
    
    if split_ratio == 1:
        tx_tr, y_tr = build_model_data_standardized(x, y)
        tx_te, y_te = 0,0
        
    else:
        x_tr, x_te, y_tr, y_te = split_data(x, y, split_ratio)
        tx_tr, y_tr = build_model_data_standardized(x_tr, y_tr)
        tx_te, y_te = build_model_data_standardized(x_te, y_te)
    
    return tx_tr, y_tr, tx_te, y_te
    
# tx, y, tx_te, y_te = load_build_split("data/train.csv", 5, 0.8)

In [4]:
# required functions
def calculate_mse(y, tx, w):
    """mean square error"""
    err = y - tx.dot(w)
    return 1/2*np.mean(err**2)

def compute_gradient(y, tx, w):
    """gradient computation for linear regression"""
    """(x transpose times w) is linear predictor"""
    err = tx.dot(w) - y
    grad = tx.T.dot(err) / len(err)
    return grad, err

In [5]:
# main functions
def least_squares_GD(y, tx, initial_w, max_iters, gamma):
    """linear regression using gradient descent"""
    ws = [initial_w]
    losses = []
    w = initial_w
    for n_iter in range(max_iters):
        # compute gradient, loss
        grad, _ = compute_gradient(y, tx, w)
        # gradient w by descent update
        w = w - gamma * grad
        # calculate loss
        loss = calculate_mse(y, tx, w)
        # store w and loss
        ws.append(w)
        losses.append(loss)
        print("GD({bi}/{ti}): loss={l}, weights={},{},{}".format(
              bi=n_iter, ti=max_iters - 1, l=loss, *w.round(5)))

    return losses, ws

In [6]:
def least_squares_SGD(y, tx, initial_w, max_iters, gamma, batch_size=1):
    """linear regression using stochastic gradient descent"""
    ws = [initial_w]
    losses = []
    w = initial_w
    for n_iter in range(max_iters):
        for y_batch, tx_batch in batch_iter(y, tx, batch_size=batch_size, num_batches=1):
            # compute a stochastic gradient and loss
            grad, _ = compute_gradient(y_batch, tx_batch, w)
            # update w through the stochastic gradient update
            w = w - gamma * grad
            # calculate loss
            loss = calculate_mse(y, tx, w)
            # store w and loss
            ws.append(w)
            losses.append(loss)

        print("SGD({bi}/{ti}): loss={l}, weights={}".format(
              bi=n_iter, ti=max_iters - 1, l=loss, *w.round(5)))

    return losses, ws

In [7]:
def least_squares(y, tx):
    """least squares regression using normal equations"""
    a = tx.T.dot(tx)
    b = tx.T.dot(y)
    w = np.linalg.solve(a, b)
    loss = calculate_mse(y, tx, w)
    return loss, w

In [8]:
# required functions
def ridge_mse(y, tx, w):
    """compute the loss by mse."""
    e = y - tx.dot(w)
    mse = e.dot(e) / (2 * len(e))
    return mse

# main function
def ridge_regression(y, tx, lambda_):
    """rige regression using normal equations"""
    aI = 2 * tx.shape[0] * lambda_ * np.identity(tx.shape[1])
    a = tx.T.dot(tx) + aI
    b = tx.T.dot(y)
    w = np.linalg.solve(a, b)
    loss = ridge_mse(y, tx, w)
    return loss, w

In [9]:
# required function(s)
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def logistic_cost(y, tx, w):    
    """cost for logistic regression """
    epsilon = 1e-8
    sig = sigmoid(tx.dot(w));
    cost = (-y) * np.log(sig + epsilon) - (1-y) * np.log(1-sig + epsilon)
    return np.mean(cost)

def logistic_gradient(y, tx, w):
    """gradient for logistic regression """
    err = sigmoid(tx.dot(w)) - y
    grad = tx.T.dot(err) / len(err)
    return grad, err

In [10]:
# main function(s)
def logistic_regression(y, tx, initial_w, max_iters, gamma):
    """logistic regression using GD"""
    ws = [initial_w]
    losses = []
    w = initial_w
    for n_iter in range(max_iters):
        # compute gradient, loss
        grad, _ = logistic_gradient(y, tx, w)
        # gradient w by descent update
        w = w - gamma * grad
        # calculate loss
        loss = logistic_cost(y, tx, w)
        # store w and loss
        ws.append(w)
        losses.append(loss)
        print("GD({bi}/{ti}): loss={l}, weights={},{},{}".format(
              bi=n_iter, ti=max_iters - 1, l=loss, *w.round(5)))

    return losses, ws

In [11]:
def logistic_regression_SGD(y, tx, initial_w, max_iters, gamma, batch_size=1):
    """logistic regression using stochastic SGD"""
    ws = [initial_w]
    losses = []
    w = initial_w
    for n_iter in range(max_iters):
        for y_batch, tx_batch in batch_iter(y, tx, batch_size=batch_size, num_batches=1):
            # compute a stochastic gradient and loss
            grad, _ = logistic_gradient(y_batch, tx_batch, w)
            # update w through the stochastic gradient update
            w = w - gamma * grad
            # calculate loss
            loss = logistic_cost(y, tx, w)
            # store w and loss
            ws.append(w)
            losses.append(loss)

        print("SGD({bi}/{ti}): loss={l}, weights={},{},{}".format(
              bi=n_iter, ti=max_iters - 1, l=loss, *w.round(5)))
    return losses, ws

In [12]:
# required function(s)
def reg_logistic_cost(y, tx, w, alpha):
    epsilon = 1e-8
    """cost for logistic regression with regularization"""
    sig = sigmoid(tx.dot(w));
    cost = (-y) * np.log(sig + epsilon) - (1-y) * np.log(1-sig + epsilon)
    reg = np.dot(w,w) * alpha / (2 * len(y))
    return np.mean(cost) + reg

def reg_logistic_gradient(y, tx, w, alpha):
    """gradient for logistic regression with with regularization"""
    err = sigmoid(tx.dot(w)) - y
    grad = tx.T.dot(err) / len(err)
    reg = w * alpha / len(err)
    return grad - reg, err

In [13]:
# main function(s)
def reg_logistic_regression(y, tx, alpha, initial_w, max_iters, gamma):
    """regularized logistic regression using GD"""
    ws = [initial_w]
    losses = []
    w = initial_w
    for n_iter in range(max_iters):
        # compute gradient, loss
        grad, _ = reg_logistic_gradient(y, tx, w, alpha)
        # gradient w by descent update
        w = w - gamma * grad
        # calculate loss
        loss = reg_logistic_cost(y, tx, w, alpha)
        # store w and loss
        ws.append(w)
        losses.append(loss)
    
    '''
    print("GD({bi}/{ti}): loss={l},\
          \nweights={},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}"
          .format(bi=n_iter, ti=max_iters - 1, l=loss, *w.round(2)))
    '''
    
    print("GD({bi}/{ti}): loss={l}".format(bi=n_iter, ti=max_iters - 1, l=loss,))
    
    return losses, ws

In [14]:
def test_reg_logistic_regresion(tx_tr, y_tr, tx_te, y_te, alpha, max_iters, gamma):
    n = num_features = len(tx_tr.T)
    initial_w = np.random.randn(n)
#   initial_w = np.zeros(n)
    losses, ws = reg_logistic_regression(y_tr, tx_tr, alpha, initial_w, max_iters, gamma)

    # accuracy
    y_pred = predict_labels_test(ws[-1], tx_te)
    return ws[-1], (1 - (abs(y_pred - y_te).sum() / len(y_te)))

In [15]:
def reg_logistic_regression_SGD(y, tx, alpha, initial_w, max_iters, gamma, batch_size=1):
    """regularized logistic regression using SGD"""
    ws = [initial_w]
    losses = []
    w = initial_w
    for n_iter in range(max_iters):
        for y_batch, tx_batch in batch_iter(y, tx, batch_size=batch_size, num_batches=1):
            # compute a stochastic gradient and loss
            grad, _ = reg_logistic_gradient(y_batch, tx_batch, w, alpha)
            # update w through the stochastic gradient update
            w = w - gamma * grad
            # calculate loss
            loss = reg_logistic_cost(y, tx, w, alpha)
            # store w and loss
            ws.append(w)
            losses.append(loss)

        print("SGD({bi}/{ti}): loss={l}, weights={},{},{}".format(
              bi=n_iter, ti=max_iters - 1, l=loss, *w.round(5)))
    return losses, ws